# extração do texto do PDF

In [20]:
import PyPDF2
import nltk

In [101]:
#Listagem de palavras frequentes que não alteram o significado do texto (em ingles)
nltk.download('stopwords')
from nltk.corpus import stopwords
irrelevantes = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rafae\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [102]:
#tratamento de flexão de palavras por lematização
tk = nltk.tokenize.WhitespaceTokenizer()
nltk.download('wordnet')
nltk.download('omw-1.4')
lmt = nltk.stem.WordNetLemmatizer()

#A biblioteca nltk não tem dados para lematização em PT
def lematize_text(text):
    lem_words = list()
    for word in tk.tokenize(text):
        lem_words.append(lmt.lemmatize(word))
    return lem_words


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\rafae\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\rafae\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [89]:
reader = PyPDF2.PdfReader('ArquivosEN/1608.06902.pdf')

In [92]:
len(reader.pages)

11

In [95]:
textin = reader.pages[0].extract_text()

In [100]:
lematize_text(textin)

['Recurrent',
 'Neural',
 'Networks',
 'With',
 'Limited',
 'Numerical',
 'Precision',
 'Joachim',
 'Ott\x03,',
 'Zhouhan',
 'Linz,Ying',
 'Zhangz,',
 'Shih-Chii',
 'Liu\x03,',
 'Yoshua',
 'Bengiozy',
 '\x03Institute',
 'of',
 'Neuroinformatics,',
 'University',
 'of',
 'Zurich',
 'and',
 'ETH',
 'Zurich',
 'ottj@ethz.ch,',
 'shih@ini.ethz.ch',
 'zDépartement',
 'd’informatique',
 'et',
 'de',
 'recherche',
 'opérationnelle,',
 'Université',
 'de',
 'Montréal',
 'yCIFAR',
 'Senior',
 'Fellow',
 '{zhouhan.lin,',
 'ying.zhang}@umontreal.ca',
 'Abstract',
 'Recurrent',
 'Neural',
 'Networks',
 '(RNNs)',
 'produce',
 'state-of-art',
 'performance',
 'on',
 'many',
 'machine',
 'learning',
 'task',
 'but',
 'their',
 'demand',
 'on',
 'resource',
 'in',
 'term',
 'of',
 'memory',
 'and',
 'computational',
 'power',
 'are',
 'often',
 'high.',
 'Therefore,',
 'there',
 'is',
 'a',
 'great',
 'interest',
 'in',
 'optimizing',
 'the',
 'computation',
 'performed',
 'with',
 'these',
 'model',
